# Actions from Trello

This notebook tests pulling data from Trello via the trello APIs.  

It pulls back all actions over a certain period and builds a pandas dataframe with the data for a cumulative flow diagram in Plotly (so we can render in dash ultimately)  

In [ ]:
import requests
import json
import pandas as pd
import os
from requests_oauthlib import OAuth1Session

Pull in all variables - these are stored as secrets and defined either in the project or passed in to codespaces when it is started. 

In [ ]:
trello_key = os.getenv('TRELLO_KEY')
trello_token = os.getenv('TRELLO_TOKEN')
trello_board_id = os.getenv('TRELLO_BOARD_ID_SHORT')

Run the query against the trello api endpoints. 

In [ ]:
uri = f"https://api.trello.com/1/boards/{trello_board_id}/actions"
filter = "since=Jun%201%202023&limit=1000&filter=createCard,updateCard"
url = uri + "?" + filter
oauthRequest = OAuth1Session(trello_key,
                    resource_owner_key=trello_token)

headers = {'Accept': "application/json"}
r = oauthRequest.get(url, headers=headers)
print(r.status_code)

Extract the results in json

In [ ]:
json_data = json.loads(r.text)

print(len(json_data))
print(json.dumps(json_data, indent = 2, sort_keys=True))

Flatten (normalise) the nested json to the bits that matter

In [ ]:
df = pd.json_normalize(json_data)

In [ ]:
df

In [ ]:
df.to_csv("output.csv")

Filter to just show the last 1 most recent action per card based on a specific date. 

TODO: Count the number of cards per list


TODO: Save into a dataframe


TODO: Render the data in a plotly diagram

TODO: Iterate over generate data for every day


TODO: Render the data in a plotly diagram

TODO: Iterate filling over the data from all time & rendering into a plotly diagram

Troubleshooting: Dump output of json into a pretty nested format. 

In [ ]:
print(json.dumps(json_data, indent = 2, sort_keys=True))

Pull back a subset of the data that we care about

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.info()



In [ ]:
df.set_index('date', inplace=True)
df.info()

In [ ]:
df = df.tz_convert("Pacific/Auckland")
df.info()

In [ ]:
df[["id",
    "data.card.name",
    "type",
    "data.card.closed",
    "data.list.name",
    "data.listBefore.name",
    "data.listAfter.name",
    "appCreator.id",
    "appCreator.name",
    "data.card.idLabels"]]